In [1]:
# jesseLiu2000
from unicodedata import bidirectional
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.embedding = nn.Embedding(n_seq, embedding_size)
        self.bilstm = nn.LSTM(input_size=embedding_size, hidden_size=n_hidden, bidirectional=True, num_layers=4)
        self.fc1 = nn.Linear(n_hidden*2, n_seq)

    def forward(self, state, X):
        X1 = self.embedding(X)
        X1 = X1.float()
        X1 = X1.permute(1, 0, 2)
        LSTM_X, state = self.bilstm(X1, state)
        LSTM_X = LSTM_X[-1]  
        output = self.fc1(LSTM_X)
        return output


if __name__ == '__main__':
    embedding_size = 2
    n_hidden = 5

    file_list = ['He hate milk', 'Its cold today', 'I like cat']
    batch_size = len(file_list)

    word_vocab = ' '.join(file_list).split()
    word_vocab = list(set(word_vocab))
    word2index = {w: i for i, w in enumerate(word_vocab)}
    index2word = {i: w for i, w in enumerate(word_vocab)}

    n_seq = len(word2index)

    input = [i.split(' ')[:-1] for i in file_list]
    map_input = [[word2index[i] for i in j] for j in input]
    target = [i.split(' ')[-1] for i in file_list]
    map_target = [word2index[i] for i in target]

    model = TextLSTM()
    lr = 1e-3
    epoch = 5000
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    inputs = torch.LongTensor(map_input)
    targets = torch.LongTensor(map_target)

    # Training
    for ep in range(epoch):
        optimizer.zero_grad()

        state = None
        output = model(state, inputs)

        loss = criterion(output, targets)
        if (ep + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (ep + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    input = [sentence.split()[:2] for sentence in file_list]

    # Test
    state = None
    predict = model(state, inputs).data.max(1, keepdim=True)[1]
    print("Input:", [sentence.split()[:2] for sentence in file_list])
    print("Output:", [index2word[word.item()] for word in predict.squeeze()])

Epoch: 1000 cost = 0.036333
Epoch: 2000 cost = 0.007272
Epoch: 3000 cost = 0.003088
Epoch: 4000 cost = 0.001579
Epoch: 5000 cost = 0.000867
Input: [['He', 'hate'], ['Its', 'cold'], ['I', 'like']]
Output: ['milk', 'today', 'cat']
